In [5]:
import tkinter as tk
from tkinter import scrolledtext
import spacy
from spacy import displacy
import matplotlib.pyplot as plt
import networkx as nx
import snakes.plugins
snakes.plugins.load("gv", "snakes.nets", "nets")
from nets import *

nlp = spacy.load("en_core_web_sm")


In [6]:
together_words = {"and", "both", "along with", "as well as", "in conjunction with", "together with"}
individual_words = {"or", "either", "one of", "neither", "nor", "any of", "each of"}

In [7]:
#Entities
class Preposition:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return f"({self.name})"
    
    
class Verb:
    def __init__(self, name):
        self.name = name
    def __str__(self):
        return f"{self.name}"

class Noun:
    def __init__(self, name, preposition=None):
        self.name = name
        self.preposition = preposition if isinstance(preposition, list) else [preposition]
    
    def __str__(self):
        if self.preposition:
            p = ', '.join(str(p) for p in self.preposition)
            return f"{self.name}, {p}"
        else:
            return f"{self.name}"

    
class Task:
    def __init__(self, verbs, nouns):
        self.verbs = verbs if isinstance(verbs, list) else [verbs] 
        self.nouns = nouns if isinstance(nouns, list) else [nouns]
    def __str__(self):
        verb_str = ', '.join(str(verbs) for verbs in self.verbs)
        nouns_str = ', '.join(str(noun) for noun in self.nouns)
        return f"Task(Verbs: [{verb_str}], Nouns: [{nouns_str}])"
    
    def get_all_actors(self):
        actors = []
        for noun in self.nouns:
            actors.extend(noun.name)  
        return actors


preposition1 = Preposition("on")
noun1 = Noun("table", preposition1)
noun2 = Noun("chair")

# Create a list of verbs
verbs = Task(["places", "moves"], [noun1, noun2])

# Print out the verb and its related nouns and prepositions
print(verbs)  # Output: Verb(places, moves, Noun(table, floor, Preposition(on)), Noun(chair))


Task(Verbs: [places, moves], Nouns: [table, (on), chair, None])


In [8]:
def analyse_tree(doc):

    all_tasks = []

    def add_actors(subj):
        prepositions = []
        collect_prepositions(subj, prepositions)
        actor = Noun(subj.text, prepositions)
        return actor
    
    def add_task(verbs, actors):
        task = Task(verbs, actors)
        return task
    
    def add_verb(verb):
        verb1 = Verb(verb.text)
        return verb1

    def collect_subj_conjuncts(subj, actors):
        for child in subj.children:
            if child.dep_ == "conj":
                actor = add_actors(child)
                actors.append(actor)
                collect_subj_conjuncts(child, actors)
    
    def collect_prepositions(subj, prepositions):
        for child in subj.children:
            if child.dep_ == "cc":
                preposition = Preposition(child.text)
                prepositions.append(preposition)


    def collect_conjuncts(verb, root_subject, verb_data):
        for child in verb.children:
            if child.dep_ == "conj" and child.pos_ == "VERB":
                conj_nsubj = []

                # Check if the conjunct verb has its own subject
                for grandchild in child.children:
                    if grandchild.dep_ == "nsubj":
                        actor = add_actors(grandchild)
                        conj_nsubj.append(actor)
                        collect_subj_conjuncts(grandchild, conj_nsubj)

                # If no subject for the conjunct verb, add it to the root's verb_data
                if not conj_nsubj:
                    verb = add_verb(child)
                    verb_data.append(verb) 
                else:
                    verb = add_verb(child)
                    task = add_task(verb, conj_nsubj)
                    all_tasks.append(task)

                collect_conjuncts(child, root_subject, verb_data)

    for token in doc:
        if token.dep_ == "ROOT":
            verb = add_verb(token)
            verb_data = [verb]  # Start with the root verb
            actors_list = []

            for child in token.children:
                if child.dep_ == "nsubj":
                    actor = add_actors(child)
                    actors_list.append(actor)
                    collect_subj_conjuncts(child, actors_list)

            task = add_task(verb_data, actors_list)
            all_tasks.append(task)
            collect_conjuncts(token, actors_list, verb_data)

    return all_tasks


In [9]:
def is_or_exist(actors):
    for actor in actors:
        for p in actor.preposition:
            if p.name == "or":
                return True
    return False

def is_this_or(actor):
    for p in actor.preposition:
        if p.name == "or":
            return True
    return False

def is_and_exist(actors):
    for actor in actors:
        for p in actor.preposition:
            if p.name == "and":
                return True
    return False

def print_actors_names(actors):
    s = ""
    for a in actors:
        s+= a.name+("_")
    return s

#rules for created dictionary
def get_parallel_tasks(doc):
    order_cnt = 0
    actors_actions = {}
    all_tasks = analyse_tree(doc)
    print(all_tasks)

    for task in all_tasks:
        if len(task.nouns) > 1:
            for verb in task.verbs:

                #task choice
                if (is_or_exist(task.nouns)): #gather actors groups if there is OR 
                    actors_before = []
                    groups = ""
                    for actor in task.nouns:
                        if actor.name not in actors_actions:
                            actors_actions[actor.name] = []
                            actors_before.append(actor)
                        if (is_this_or(actor)):
                            if (is_and_exist(actors_before)):
                                names_str = print_actors_names(actors_before)
                                for a in actors_before:
                                    actors_actions[a.name].append("OR_" + names_str + str(order_cnt))
                                    actors_actions[a.name].append(verb.name)
                            else:
                                for a in actors_before:
                                    actors_actions[a.name].append("OR_" + a.name + "_" + str(order_cnt))
                                    actors_actions[a.name].append(verb.name) 
                            actors_before = []                           
                    names_str = print_actors_names(actors_before)
                    for a in actors_before:
                        actors_actions[a.name].append("OR_" + names_str + str(order_cnt))
                        actors_actions[a.name].append(verb.name)
                    print(groups)
                    order_cnt+=1
                    
                #no choice
                else:
                    for actor in task.nouns:                                               
                        if actor.name not in actors_actions:
                            actors_actions[actor.name] = []
                        names_str = print_actors_names(task.nouns)
                        actors_actions[actor.name].append("BARRIER_" + names_str + str(order_cnt))
                        actors_actions[actor.name].append(verb.name)
                        actors_actions[actor.name].append("SINC_"+ names_str + str(order_cnt))
                    order_cnt+=1
        else:
            actor = task.nouns[0]
            if actor.name not in actors_actions:
                    actors_actions[actor.name] = []
            for verb in task.verbs:
                    actors_actions[actor.name].append(verb.name)
    print(actors_actions)
    return actors_actions

input_text ="Robot A, Robot B and Robot C walk."
doc = nlp(input_text)
get_parallel_tasks(doc)



{'A': ['BARRIER_A_B_C_0', 'walk', 'SINC_A_B_C_0'], 'B': ['BARRIER_A_B_C_0', 'walk', 'SINC_A_B_C_0'], 'C': ['BARRIER_A_B_C_0', 'walk', 'SINC_A_B_C_0']}


{'A': ['BARRIER_A_B_C_0', 'walk', 'SINC_A_B_C_0'],
 'B': ['BARRIER_A_B_C_0', 'walk', 'SINC_A_B_C_0'],
 'C': ['BARRIER_A_B_C_0', 'walk', 'SINC_A_B_C_0']}

In [10]:
n = PetriNet('mynet')
n.add_place(Place('p', [0]))
n.add_transition(Transition('t', Expression('x<5')))
n.add_input('p', 't', Variable('x'))
n.add_output('p', 't', Expression('x+1'))
n.draw("value-1.png")

Graph(['node_0', 'node_1'], [])

In [13]:
# Test
#OR cases

#successful
#input_text = "Mary and Ben or Mark and John walk."
#input_text = "Mary, Ben or Mark and John walk."
#input_text = "Mary, Ben or Mark walk."
input_text = "Robot A and Robot B, or Robot A and Robot C walk."


doc = nlp(input_text)
get_parallel_tasks(doc)


{'A': ['OR_A_B_0', 'walk'], 'B': ['OR_A_B_0', 'walk'], 'C': ['OR_C_0', 'walk']}


{'A': ['OR_A_B_0', 'walk'], 'B': ['OR_A_B_0', 'walk'], 'C': ['OR_C_0', 'walk']}

In [12]:
# Test

#input_text = "Robot A and Robot B take the pot. Robot C takes the spoon. Robots A and Robot C help each other"
#input_text = "Robot A takes the pan, Robot B takes the pencil."
#input_text = "Robot A takes the pan and drops it. "
#input_text = "Robot A, Robot B and Robot C takes the pan, Robot B takes the pencil"
#input_text = "Robot A and Robot B take the pot. Robot C takes the spoon. Robot A and Robot C work together and Robot B is cleaning"

#WEIRD BEHAVIOUR in spacy
#input_text = "Robots A and Robot C help each other and Robot B is cleaning"
#input_text = "Robots A and Robot C help each other, Robot B cleans"

input_text = "Robot A goes."

doc = nlp(input_text)
all_tasks = analyse_tree(doc)
for i in all_tasks:
    print(i)

#options={"compact": True, "distance":60}
spacy.displacy.serve(doc, style="dep", auto_select_port=True)
#print_tasks(doc)


Task(Verbs: [goes], Nouns: [A])


/opt/anaconda3/lib/python3.11/site-packages/spacy/util.py:1835: UserWarning: [W124] 0.0.0.0:5000 is already in use, using the nearest available port 5001 as an alternative.
  warnings.warn(Warnings.W124.format(host=host, port=start, serve_port=port))
/opt/anaconda3/lib/python3.11/site-packages/spacy/displacy/__init__.py:106: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5001 ...

Shutting down server on port 5001.


In [ ]:
def display_input():
    input_text = entry.get()
    label.config(text=input_text)
    
    doc = nlp(input_text)
    result_text.delete(1.0, tk.END)
    
   
    subjects_info = analyse_tree(doc)
    # Insert the subjects and conjuncts information into the text area
    result_text.insert(tk.END, subjects_info)

In [ ]:
# Create the main window
root = tk.Tk()
label = tk.Label(root, text="Your input will appear here", font=('Arial', 14))
label.pack(pady=10)
entry = tk.Entry(root, font=('Arial', 14))
entry.pack(pady=10)
button = tk.Button(root, text="Analyze", command=display_input, font=('Arial', 14))
button.pack(pady=10)
result_text = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=60, height=15, font=('Arial', 12))
result_text.pack(pady=10)

root.mainloop()

# Part 1: Foundations of infovis 🚀

Let's get started with exploring some datasets with basic chart types!

👉 This part has two TODOs (1.1, 1.2).

To get started, here are the references for the data we use for some examples: 

- "Global air quality data provided by the World Health Organization" | [Source: WHO](https://www.who.int/data/gho/data/themes/air-pollution/who-air-quality-database)

- "WHO regional groupings" by income | [Source: WHO](https://cdn.who.int/media/docs/default-source/air-pollution-documents/air-quality-and-health/country-groupings-database-2022.pdf)

Let's load the dataset and have a first look: